In [1]:
import tflearn
from PIL import Image
import numpy as np

# Residual blocks
# 32 layers: n=5, 56 layers: n=9, 110 layers: n=18
n = 18

In [2]:
# Data loading
from tflearn.datasets import cifar10
(X, Y), (testX, testY) = cifar10.load_data()
Y = tflearn.data_utils.to_categorical(Y, 10)
testY = tflearn.data_utils.to_categorical(testY, 10)

100.0% 170500096 / 170498071


('Succesfully downloaded', 'cifar-10-python.tar.gz', 170498071, 'bytes.')
File Extracted in Current Directory


In [3]:
# Real-time data preprocessing
img_prep = tflearn.ImagePreprocessing()
img_prep.add_featurewise_zero_center(per_channel=True)

# Real-time data augmentation
img_aug = tflearn.ImageAugmentation()
img_aug.add_random_flip_leftright()
img_aug.add_random_crop([32, 32], padding=4)

In [4]:
# Building Residual Network
net = tflearn.input_data(shape=[None, 32, 32, 3],
                     data_preprocessing=img_prep,
                     data_augmentation=img_aug)
net = tflearn.conv_2d(net, 16, 3, regularizer='L2', weight_decay=0.0001)
net = tflearn.residual_block(net, n, 16)
net = tflearn.residual_block(net, 1, 32, downsample=True)
net = tflearn.residual_block(net, n-1, 32)
net = tflearn.residual_block(net, 1, 64, downsample=True)
net = tflearn.residual_block(net, n-1, 64)
net = tflearn.residual_block(net, 1, 64, downsample=True)
net = tflearn.residual_block(net, n-1, 64)

net = tflearn.batch_normalization(net)
net = tflearn.activation(net, 'relu')
output_layer = tflearn.global_avg_pool(net)

# Regression
net = tflearn.fully_connected(output_layer, 10, activation='softmax')
mom = tflearn.Momentum(0.1, lr_decay=0.1, decay_step=32000, staircase=True)
net = tflearn.regression(net, optimizer=mom,
                     loss='categorical_crossentropy')
# Training
model = tflearn.DNN(net, checkpoint_path='resnet_v1_152.ckpt',
                max_checkpoints=10, tensorboard_verbose=0,
                clip_gradients=0.)

Instructions for updating:
Use tf.initializers.variance_scaling instead with distribution=uniform to get equivalent behavior.
Instructions for updating:
keep_dims is deprecated, use keepdims instead


In [5]:
model.fit(X, Y, n_epoch=200, validation_set=(testX, testY),
      snapshot_epoch=False, snapshot_step=500,
      show_metric=True, batch_size=128, shuffle=True,
      run_id='resnet_cifar10')

Training Step: 108  | total loss: 1.72036 | time: 64.102s
| Momentum | epoch: 001 | loss: 1.72036 - acc: 0.3350 -- iter: 13824/50000


KeyboardInterrupt: 

In [ ]:
model.save('./resnet_v1_152.ckpt')

In [ ]:
#---------------
# now extract the penultimate hidden layer's image feature
img = Image.open(file_path)
img = img.resize((32, 32), Image.ANTIALIAS)
img = np.asarray(img, dtype="float32")
imgs = np.asarray([img])
model_test = tflearn.DNN(output_layer, session = model.session)
model_test.load('resnet_v1_152.ckpt', weights_only = True)

predict_y = model_test.predict(imgs)

print('layer\'s feature'.format(predict_y))